In [ ]:
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense
from tensorflow.keras.layers import Flatten, LSTM
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input
# from tensorflow.keras.layers.merge import Concatenate
from tensorflow.keras.optimizers import Adam

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
toxic_comments = pd.read_csv("../input/comment/1613373921-5e748a2d5fc288e9f69c5f86.csv")

In [ ]:
toxic_comments.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
print(toxic_comments.shape)

(159571, 8)


In [ ]:
filter = toxic_comments["comment_text"] != ""
toxic_comments = toxic_comments[filter]
toxic_comments = toxic_comments.dropna()

In [ ]:
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [ ]:
X = []
sentences = list(toxic_comments["comment_text"])
for sen in sentences:
    X.append(preprocess_text(sen))

In [ ]:
toxic = np.sum(toxic_comments[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]],axis = 1) > 0

In [ ]:
from nltk.corpus import stopwords

In [ ]:
my_stopwords = stopwords.words('english')

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open('../input/glove-global-vectors-for-word-representation/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

### Vectorize the comments

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words = my_stopwords)
bag_of_words = vectorizer.fit_transform(X)

In [ ]:
print("Sparsity of bag_of_words:",
      len(bag_of_words.data)/(bag_of_words.shape[0]*bag_of_words.shape[1])*100)

Sparsity of bag_of_words: 0.016165423205353118


In [ ]:
bag_of_words

<159571x168645 sparse matrix of type '<class 'numpy.float64'>'
	with 4350253 stored elements in Compressed Sparse Row format>

In [ ]:
list(vectorizer.vocabulary_.keys())

['explanation',
 'edits',
 'made',
 'username',
 'hardcore',
 'metallica',
 'fan',
 'reverted',
 'vandalisms',
 'closure',
 'gas',
 'voted',
 'new',
 'york',
 'dolls',
 'fac',
 'please',
 'remove',
 'template',
 'talk',
 'page',
 'since',
 'retired',
 'aww',
 'matches',
 'background',
 'colour',
 'seemingly',
 'stuck',
 'thanks',
 'january',
 'utc',
 'hey',
 'man',
 'really',
 'trying',
 'edit',
 'war',
 'guy',
 'constantly',
 'removing',
 'relevant',
 'information',
 'talking',
 'instead',
 'seems',
 'care',
 'formatting',
 'actual',
 'info',
 'make',
 'real',
 'suggestions',
 'improvement',
 'wondered',
 'section',
 'statistics',
 'later',
 'subsection',
 'types',
 'accidents',
 'think',
 'references',
 'may',
 'need',
 'tidying',
 'exact',
 'format',
 'ie',
 'date',
 'etc',
 'one',
 'else',
 'first',
 'preferences',
 'style',
 'want',
 'let',
 'know',
 'appears',
 'backlog',
 'articles',
 'review',
 'guess',
 'delay',
 'reviewer',
 'turns',
 'listed',
 'form',
 'eg',
 'wikipedia',
 

In [ ]:
len(list(vectorizer.vocabulary_.keys()))

168645

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words = my_stopwords)
bag_of_words = vectorizer.fit_transform(X)

In [ ]:
print(bag_of_words.shape, len(vectorizer.vocabulary_))

(159571, 168645) 168645


In [ ]:
embeds = []
zeros_embed = 100*[0]
for key,val in sorted(vectorizer.vocabulary_.items(), key = lambda kv:(kv[1], kv[0])):
# vectorizer.vocabulary_.keys():
    if key in embeddings_dictionary:
        embeds.append(embeddings_dictionary[key])
    else:
        embeds.append(zeros_embed)
    if val%1000 == 0:
        print(val,key)


0 aa
1000 accorta
2000 adminnaizsi
3000 aggrevation
4000 albo
5000 altor
6000 andjapan
7000 antiquary
8000 aquainted
9000 arrogans
10000 associazioni
11000 austrolestes
12000 baceline
13000 bannock
14000 bd
15000 benegiciary
16000 bidhonene
17000 blakey
18000 boilers
19000 bpm
20000 bronze
21000 burgnich
22000 calvary
23000 carpelan
24000 ceirtain
25000 charge
26000 chlorophyta
27000 civillist
28000 coauthor
29000 commeneced
30000 coneheads
31000 contflict
32000 correspondant
33000 creb
34000 cudnt
35000 dainomite
36000 deadend
37000 delamarche
38000 derrier
39000 dialectal
40000 diry
41000 distant
42000 donkeykong
43000 drowning
44000 eachothers
45000 egad
46000 emaneuel
47000 enigmasoldier
48000 ern
49000 euwe
50000 experimental
51000 falcionek
52000 feckless
53000 finalist
54000 flunk
55000 fozzy
56000 ftrl
57000 gallium
58000 generated
59000 gilliam
60000 golat
61000 gratim
62000 guidelinesthe
63000 halab
64000 harvested
65000 helicopterllama
66000 hiineedrequestforcomment
67000 ho

In [ ]:
import scipy.sparse as sparse
my_embeddings = sparse.csr_matrix(embeds)

In [ ]:
doc_embeds = bag_of_words.dot(my_embeddings)

In [ ]:
doc_embeds.shape

(159571, 100)

In [ ]:
X_train_de, X_test_de, y_train_de, y_test_de = train_test_split(doc_embeds, toxic, test_size=0.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import plot_roc_curve

In [ ]:
lr = LogisticRegression()
dt = DecisionTreeClassifier(max_depth=25)
rf = RandomForestClassifier(max_depth=15, n_estimators=40)
clf = SVC(kernel='linear',gamma='auto',degree=2)

In [ ]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
model = LogisticRegression(max_iter=1000)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
space = dict()
space['fit_intercept'] = [True, False]
space['C']= np.linspace(1, 10, 10)
search = RandomizedSearchCV(model, space, scoring='recall', n_jobs=1, cv=cv)
result = search.fit(X_train_de, y_train_de)
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.5476605288942155
Best Hyperparameters: {'fit_intercept': False, 'C': 7.0}

In [ ]:
model = RandomForestClassifier()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
space = dict()
space['max_depth'] = [5,6,7,8,9,10,11,12,13,14,15]
space['n_estimators']= [10,20,30,40,50]
search = RandomizedSearchCV(model, space, scoring='recall', n_jobs=-1, cv=cv)
result = search.fit(X_train_de,y_train_de)
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.56676578668789757 Best Hyperparameters: {'max_depth': 15, 'n_estimators': 40}

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
model = GaussianNB()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
space = dict()
space['var_smoothing'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1,1e-6,1e-7,1e-8,1e-9,1e-10,1e-11,1e-12,1e-13,1e-14]
search = GridSearchCV(model, space, scoring='recall', n_jobs=-1, cv=cv)
result = search.fit(X_train_de, y_train_de)
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.5367545446 Best Hyperparameters: {'var_smoothing': 1e-3}

In [ ]:
model = svm.SVC()
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)
space = dict()
space['kernel'] = ['rbf','linear','poly']
search = GridSearchCV(model, space, scoring='recall', n_jobs=-1, cv=cv)
result = search.fit(X_train_de, y_train_de)
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.595645577546478 Best Hyperparameters: {'kernel': 'linear'}

In [ ]:
model = DecisionTreeClassifier()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
space = dict()
space['max_depth']= [5,10,15,20,25,30]
search = GridSearchCV(model, space, scoring='recall', n_jobs=-1, cv=cv)
result = search.fit(X_train_res, Y_train_res)
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.5367545446 Best Hyperparameters: {'max_depth': 25}